In [1]:
#pip install scipy
#pip install shapely
#pip install dash
#pip install networkx
#pip install scikit-learn

# 🚀 Experiment Execution Flow

In [2]:
import sys
from pathlib import Path

base_dir = Path.cwd().parent
src_path = base_dir / "src"

if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

    
from project_setup import (
    setup_paths,
    load_experiments_dict,
    import_and_reload_modules
)

# Set paths and load experiments
base_dir, src_path = setup_paths()
experiments_definition = load_experiments_dict("experiments_config.json")

# Import project classes
WorldModel, DisplayManager, MatrixSimilarity, NetworkManager = import_and_reload_modules()

# Plotly settings
import plotly.io as pio
pio.renderers.default = "iframe_connected"

# Experiment configuration
experiment_id = "WS16"
experiment_display_scale = {
    "ECIR19": 0.28,
    "ECSS18": 0.15,
    "ICCSS17": 0.3,
    "WS16": 0.15
}

# Initialize world & displayer
world = WorldModel(experiment_id, experiments_definition, base_dir)
world.initialize()

Displayer = DisplayManager(world)


✅ Loaded experiment definition from experiments_config.json
✅ Loaded contact data file: tij_with_readers_WS16.dat
✅ Converted and renamed 'readers' column to 'signature'.
✅ Loaded periods file: periodes_WS16.dat
🔄 Synchronizing periods for WS16 (offset: 1h)...
✅ Loaded plan 'Foyer' (3309x2339) with 3 readers
✅ Loaded plan 'MainRoom' (3309x2339) with 2 readers
✅ Loaded plan 'PosterRoomStairs' (3309x2339) with 1 readers
✅ Loaded plan 'PosterRoom' (3309x2339) with 2 readers
✅ Computed 37 signatures.
✅ DisplayManager initialized with Plotter, Matrixer, and Networker.
✅ Mapper initialized with 4 plans.


## 🔄 Period Detection & Adjustment Pipeline

In [29]:
# 🔍 Detect Transitions (Activity-based)
debug_data = world.period_manager.detect_transitions(
    threshold_pos=0.1, threshold_neg=-0.05, smooth_sigma=2, freq="1min", debug=True
)
if debug_data:
    Displayer.plotter.plot_transition_debug_activity(
        activity_series=debug_data["activity_series"],
        smoothed=debug_data["smoothed"],
        norm_derivative=debug_data["norm_derivative"],
        transition_df=debug_data["transition_df"],
        experiment_id=world.experiment_id,
        smooth_sigma=debug_data["smooth_sigma"],
    )

🔍 Detecting transitions for WS16...
✅ Detected 49 transitions.
🛠️ Adjusting periods with transitions for WS16...
✅ Final periods updated: 32 entries.
📈 Debug Plot: Raw, Smoothed & Derivative


In [4]:
simil = MatrixSimilarity()
activity_signature = simil.compute_activity_matrix(
    world.period_manager.final_period_df, world.signatures
)

cosine_signature, signature_group = simil.compute_cosine_similarity(
    activity_input=activity_signature,
    reordered=True,
    num_groups=8,
    return_groups=True,
)
world.define_areas_by_group(signature_group)


In [5]:
world.assign_agents_to_areas_over_time(freq="20s")

✅ Assigned 138 agents with position over time.


In [6]:
world.compute_active_agent_to_area()

In [7]:
Networker = NetworkManager()
Networker.build_area_transition_graph(world)
Networker.build_temporal_area_transition_graphs(world)

In [8]:
Networker.build_temporal_agent_graphs_by_area(world,'20s')

✅ Built fixed-window temporal contact graphs in all Area objects.


In [9]:
Displayer.mapper.display(scale=experiment_display_scale[experiment_id])

In [10]:
Displayer.networker.plot_node_edge_distribution_per_area(world)

In [11]:
Displayer.networker.plot_area_transition_sankey(Networker.area_transition_graph,world)

In [12]:
world.compute_agents_entropies()

✅ Computed entropy for 138 agents.


In [13]:
Displayer.plotter.plot_agent_entropy_distribution(world.agents)

In [14]:
# === 🔄 Networker Method ===
Displayer.networker.run_dash(Networker.area_transition_graph)

In [24]:
# === 📈 Plotter Methods ===
Displayer.plotter.plot_normalized_activity(
    entities=world.areas,
    experiment_id=world.experiment_id,
    mode_label="areas",
    title_suffix="",
    show_total=True,
    sort_by_activity=True,
    df_period=world.period_manager.final_period_df,
)

In [25]:
Displayer.plotter.plot_signature_distributions()

In [27]:
Displayer.plotter.periods_df = world.period_manager.final_period_df
Displayer.plotter.report_world_model()


🧭 WorldModel Report — Experiment: WS16
═══════════════════════════════════════════════
📊 Contacts: 153371 rows × 5 columns
🗺️ Plans loaded: 4
📡 Readers registered: 8
🔗 Signatures computed: 37
═══════════════════════════════════════════════

🔎 Sample Data (df.head()):


t     i     j signature            datetime
0  1480486100   598  1492     [005] 2016-11-30 06:08:20
1  1480486100  1159  1312     [005] 2016-11-30 06:08:20
2  1480486100  1168  1492     [005] 2016-11-30 06:08:20
3  1480486120  1159  1492     [005] 2016-11-30 06:08:40
4  1480486160   598  1492     [005] 2016-11-30 06:09:20


        ### 🧪 Experiment Summary — WS16
    
        | Metric                            | Value                       |
        |------------------------------------|-----------------------------|
        | 📊 Total contacts                  | `153371`          |
        | 📡 Unique readers                  | `8`      |
        | 🔁 Avg readers per signature       | `3.20` |
        | 🔼 Max readers in one signature    | `5`             |
        | 🔽 Min readers in one signature    | `1`             |
        | 📡 Unique signatures               | `37`       |
    
        **Reader IDs:**  
        `005, 013, 014, 015, 016, 074, 075, 078`
        


✅ Report completed for WS16


In [28]:
Displayer.plotter.report_periods()


📝 Period Report — WS16
═══════════════════════════════════════════════
⏱️ Total timeline duration: 1222.00 minutes

📊 Duration breakdown by period type:
  - High Activity: 540.33 min
  - Low Activity: 555.67 min
  - Up Transitions: 59.00 min
  - Down Transitions: 67.00 min


In [22]:
Displayer.matrixer.plot_activity_matrix(
        matrix=activity_signature,
        title=f"Activity Matrix — Experiment: {experiment_id}",
        xlabel="Period",
        ylabel="Entity",
        cmap="Viridis",
        linewidth=0.5,
        linecolor="gray",
        annot=False,
        fmt=".0f",
    )


In [23]:


Displayer.matrixer.plot_cosine_similarity_matrix(
    cos_sim_df=cosine_signature,
    title=f"Cosine Similarity Between Signatures — Experiment: {experiment_id}",
    cmap="Viridis",
    linewidth=0.5,
    linecolor="gray",
    annot=False,
    fmt=".2f",
)

In [17]:
# === 🗺️ Mapper Methods ===
Displayer.mapper.display(scale=experiment_display_scale[experiment_id])

Displayer.mapper.plot_animated_signature_activity(
    grid_size=(20, 20),
    sigma=0.4,
    scale=experiment_display_scale[experiment_id],
    heatmap_opacity=0.8,
    time_bin="30min"
)
